In [1]:
# After you run this cell you must restart the sessation
!pip install langchain langchain-community
!pip install -U langchain-core
!pip install --upgrade langchain langchain-core langchain-community langchain-classic
!pip install langgraph

In [2]:
print("Installing OCR tools...")
!sudo apt-get install tesseract-ocr > /dev/null
!pip install pytesseract > /dev/null
print("Done.")

Installing OCR tools...
Done.


In [3]:
# ==============================
#  INSTALL DEPENDENCIES
# ==============================
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks
    import torch
    v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install -U bitsandbytes


### the librays

In [4]:
from unsloth import FastLanguageModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import numpy as np
import cv2
import os
from google.colab import drive
from matplotlib import pyplot as plt
from langchain_core.language_models import LLM
from typing import Any, List, Optional, Mapping
from pydantic import Field
from peft import PeftModel
from langchain_core.tools import Tool
import gradio as gr
from PIL import Image
import re
from langgraph.graph import StateGraph
from typing import TypedDict
from langchain_core.tools import Tool
import uuid
import base64
from io import BytesIO
import gradio as gr
import joblib
import pandas as pd
import pytesseract
from sklearn.preprocessing import StandardScaler

try:
    from PIL import Image
except ImportError:
    import Image

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!


## LOAD The BASE MODEL from unsloth

In [5]:
max_seq_length = 2048  # Auto support RoPE Scaling
dtype = None  # None for auto detection
load_in_4bit = True  # Use 4bit quantization

# 4-bit prequantized models
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

### LOAD LORA WEIGHTS FOR FINE-TUNING

In [6]:
LORA_MODEL = "Mohamed-Abdelsamed/llama-medical-lora"
print("Attaching LoRA weights...")
lora_model = PeftModel.from_pretrained(model, LORA_MODEL)
FastLanguageModel.for_inference(lora_model)
print("LoRA loaded successfully!")


Attaching LoRA weights...


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

LoRA loaded successfully!


### mount to GOOGLE DRIVE

In [7]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### LOAD MODEL DenseNet121

In [8]:
DRIVE_MODEL_PATH = '/content/drive/MyDrive/My_Medical_Project_Models/densenet_pneumonia_final.pth'

print("Setting up DenseNet121 architecture...")

model = models.densenet121(pretrained=False)
n_features = model.classifier.in_features
model.classifier = nn.Linear(n_features, 3)

print(f"Loading weights from: {DRIVE_MODEL_PATH}")

state_dict = torch.load(DRIVE_MODEL_PATH, map_location="cpu")
model.load_state_dict(state_dict)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

print(f"✅ Model loaded on {device}.")


# ================================
# GRAD-CAM IMPLEMENTATION
# ================================
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.activations = None

        # Forward hook to get activations
        self.target_layer.register_forward_hook(self._forward_hook)

    def _forward_hook(self, module, input, output):
        self.activations = output

    def generate_heatmap(self, input_tensor, class_idx):
        """
        input_tensor: preprocessed tensor (B,C,H,W) on device
        class_idx: index of target class
        """
        self.model.zero_grad()

        # Forward pass
        output = self.model(input_tensor)
        score = output[0, class_idx]

        # Get gradients w.r.t target layer activations
        gradients = torch.autograd.grad(outputs=score,
                                        inputs=self.activations,
                                        retain_graph=True,
                                        create_graph=False)[0]

        # Global average pooling of gradients
        weights = torch.mean(gradients, dim=(2,3), keepdim=True)

        # Weighted combination of activations
        cam = torch.sum(weights * self.activations, dim=1).squeeze()

        # ReLU + normalize
        cam = torch.relu(cam)
        cam_np = cam.detach().cpu().numpy()
        cam_np = cam_np - cam_np.min()
        if cam_np.max() != 0:
            cam_np = cam_np / cam_np.max()

        return cam_np

print(" Grad-CAM Ready.")


Setting up DenseNet121 architecture...
Loading weights from: /content/drive/MyDrive/My_Medical_Project_Models/densenet_pneumonia_final.pth
✅ Model loaded on cuda.
🔥 Grad-CAM Ready.


In [9]:
# preprocising
IMG_SIZE = 224
class_names = ["NORMAL", "PNEUMONIA", "UNKNOWN"]

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def predict_single_image(image_path, model, transform):
    try:
        img = Image.open(image_path).convert('RGB')
    except:
        print("❌ Error loading image")
        return None, None, None

    input_tensor = transform(img)
    input_batch = input_tensor.unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(input_batch)
        probs = F.softmax(output, dim=1)

    idx = torch.argmax(probs, dim=1).item()
    confidence = probs[0, idx].item()

    return idx, confidence, probs.cpu().squeeze()


In [10]:
# ==============================
# CHATBOT FUNCTION
# ==============================
system_instruction = """You are a medical AI assistant.
Always answer concisely and factually.
Never give personal opinions.
Always remind the user to consult a doctor when needed.
Do not add extra greetings or repetitive phrases."""

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
"""

def generate_answer(history, question):
    lower_question = question.lower().strip()
    greetings = ["hi", "hello", "hey", "good morning", "good afternoon", "good evening"]
    if lower_question in greetings:
        response = "Hello! I am your medical AI assistant. You can ask me any medical question."
        history.append({"role": "user", "content": question})
        history.append({"role": "assistant", "content": response})
        return history, ""
    gratitude_msgs = ["thank you", "thanks", "thanks a lot", "you helped me", "thank you very much"]
    if any(msg in lower_question for msg in gratitude_msgs):
        response = "You are welcome! I'm here to help."
        history.append({"role": "user", "content": question})
        history.append({"role": "assistant", "content": response})
        return history, ""
    instruction = system_instruction
    prompt = alpaca_prompt.format(instruction, question)
    inputs = tokenizer([prompt], return_tensors="pt").to(lora_model.device)
    outputs = lora_model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.2,
        top_p=0.8,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    stop_phrases = ["Hello", "Thank you for asking", "Welcome"]
    for phrase in stop_phrases:
        response = response.replace(phrase, "")
    response = "\n".join([line.strip() for line in response.split("\n") if line.strip()])
    history.append({"role": "user", "content": question})
    history.append({"role": "assistant", "content": response})
    return history, ""


### LOAD CBC MODEL & DEFINE DUAL-MODE OCR TOOLS

In [12]:
# --- CONFIGURATION ---
CBC_MODEL_FILE = "/content/best_model_pipeline.joblib"
RAW_DATA_FILE = "/content/raw data.csv"

# ==========================================
# 1. LOAD MODEL & SCALER
# ==========================================
print("Loading CBC Model and Scaler...")
try:
    saved_bundle = joblib.load(CBC_MODEL_FILE)
    model_pipeline = saved_bundle['pipeline']
    label_encoder = saved_bundle['label_encoder']
    required_features = saved_bundle['feature_columns']

    # Build Normalizer from Raw Data
    df_raw = pd.read_csv(RAW_DATA_FILE)
    ne = df_raw['NE#'] if 'NE#' in df_raw else df_raw.get('NE', 0)
    ly = df_raw['LY#'] if 'LY#' in df_raw else df_raw.get('LY', 0)
    df_raw['NLR'] = np.where(ly > 0, ne / ly, 0)

    for col in required_features:
        if col not in df_raw.columns:
            df_raw[col] = df_raw.select_dtypes(include=np.number).median().mean()

    scaler = StandardScaler()
    scaler.fit(df_raw[required_features])
    print(" CBC Model & Normalizer Ready.")

except Exception as e:
    print(f" Error loading CBC Model: {e}")

# ==========================================
# 2. HELPER FUNCTIONS (OCR & REGEX)
# ==========================================
def run_ocr_on_image(img_obj):
    """ Helper to run OCR on a specific image object """
    try:
        return pytesseract.image_to_string(img_obj).upper()
    except:
        return ""

def find_value_in_text(text_upper, feature, synonyms):
    """ Helper to find a specific feature in text using Regex and Sanity Checks """
    terms = synonyms.get(feature, [feature])
    for term in terms:
        # Regex: Term -> junk -> Number
        pattern = re.escape(term) + r"[^\d]{0,30}(\d+[\.]?\d*)"
        match = re.search(pattern, text_upper)
        if match:
            try:
                val_str = match.group(1)
                # Fix common OCR typos
                if val_str.count('.') > 1: val_str = val_str.rsplit('.', 1)[0]
                if val_str.startswith('.'): val_str = '0' + val_str
                val = float(val_str)

                # --- SANITY CHECKS ---
                if feature == 'HGB' and val > 50: val /= 10
                if feature == 'WBC' and val > 100: val /= 100
                if feature == 'MCV' and val > 200: val /= 10
                if feature == 'RDW':
                    if val > 100: val /= 10
                    if val > 30: return None, None
                if feature == 'MCHC' and val < 20: return None, None
                if '#' in feature and val > 20: val /= 100

                return val, term
            except: continue
    return None, None

def extract_data_dual_mode(image_path, feature_list):
    """ Runs OCR on Original AND Resized image to catch all text """
    try:
        img_orig = Image.open(image_path)
        # Create a double-size version for better small-text reading
        img_resized = img_orig.resize((img_orig.width * 2, img_orig.height * 2), Image.Resampling.LANCZOS).convert('L')
    except:
        return None

    # Scan both
    text_orig = run_ocr_on_image(img_orig)
    text_resized = run_ocr_on_image(img_resized)

    synonyms = {
        'WBC': ['WBC', 'TOTAL LEUCOCYTIC COUNT', 'TLC', 'WHITE BLOOD'],
        'RBC': ['RBC', 'TOTAL RBC', 'RED BLOOD', 'RBC COUNT'],
        'HGB': ['HGB', 'HEMOGLOBIN', 'HAEMOGLOBIN', 'HB'],
        'HCT': ['HCT', 'PCV', 'HEMATOCRIT'],
        'MCV': ['MCV', 'MEAN CORPUSCULAR VOLUME'],
        'MCH': ['MCH', 'MEAN CORPUSCULAR HEMOGLOBIN'],
        'MCHC': ['MCHC', 'MEAN CORP. HEM. CONC'],
        'RDW': ['RDW', 'RDW-CV', 'RED CELL DISTRIBUTION'],
        'PLT': ['PLT', 'PLATELET COUNT', 'THROMBOCYTE'],
        'MPV': ['MPV', 'MEAN PLATELET VOLUME'],
        'LY%': ['LY%', 'LYMPHOCYTE %'], 'MO%': ['MO%', 'MONOCYTE %'],
        'NE%': ['NE%', 'NEUTROPHIL %', 'POLY %'], 'EO%': ['EO%', 'EOSINOPHIL %'],
        'BA%': ['BA%', 'BASOPHIL %'],
        'LY#': ['LY#', 'ABS LYMPH'], 'MO#': ['MO#', 'ABS MONO'],
        'NE#': ['NE#', 'ABS NEUT'], 'EO#': ['EO#', 'ABS EOS'], 'BA#': ['BA#', 'ABS BASO']
    }

    extracted = {}
    for feature in feature_list:
        if feature == 'NLR': continue
        # Try Resized First, then Original
        val, term = find_value_in_text(text_resized, feature, synonyms)
        if val is None:
            val, term = find_value_in_text(text_orig, feature, synonyms)

        extracted[feature] = val # Can be None

    return extracted

# ==========================================
# 3. MAIN DIAGNOSIS FUNCTION (The "Tool")
# ==========================================
def cbc_diagnosis_func(image_path):
    """
    Main function called by Chatbot.
    Orchestrates OCR -> Math Fill-in -> Prediction -> Report Generation.
    """
    if not image_path: return "❌ Error: No image provided."

    # A. Extract Data (Dual Mode)
    data = extract_data_dual_mode(image_path, required_features)
    if not data: return "⚠️ Could not read image text. Please use a clearer image."

    # B. Math Fill-in (Calculate missing values)
    # Use 'extracted' values to calculate missing percentages or counts
    wbc = data.get('WBC')
    if wbc and wbc > 0:
        for cell in ['NE', 'LY', 'MO', 'EO', 'BA']:
            abs_key, pct_key = cell + '#', cell + '%'
            # Calc Pct from Abs
            if data.get(abs_key) is not None and data.get(pct_key) is None:
                data[pct_key] = round((data[abs_key] / wbc) * 100, 1)
            # Calc Abs from Pct
            elif data.get(pct_key) is not None and data.get(abs_key) is None:
                data[abs_key] = round((data[pct_key] / 100) * wbc, 2)

    # C. Calculate NLR
    ne, ly = data.get('NE#'), data.get('LY#')
    data['NLR'] = ne / ly if (ne and ly and ly > 0) else 0

    # D. Prepare Input for Model
    input_df = pd.DataFrame([data])
    for col in required_features:
        if col not in input_df.columns: input_df[col] = np.nan
    input_df = input_df[required_features].apply(pd.to_numeric, errors='coerce')

    # E. Predict
    try:
        input_values = input_df.values
        # Fill missing with scaler mean
        inds = np.where(np.isnan(input_values))
        input_values[inds] = np.take(scaler.mean_, inds[1])

        input_scaled = scaler.transform(input_values)
        pred_idx = model_pipeline.predict(input_scaled)[0]
        diagnosis = label_encoder.inverse_transform([pred_idx])[0]

        if hasattr(model_pipeline, "predict_proba"):
            confidence = np.max(model_pipeline.predict_proba(input_scaled)) * 100
        else:
            confidence = 0.0

        # F. Format the Report for the Chatbot (Markdown)
        report = f"###  CBC Analysis Report\n"
        report += f"**Diagnosis:** {diagnosis}\n"
        if confidence > 0:
            report += f"**Confidence:** {confidence:.1f}%\n\n"

        report += "| Feature | Value |\n|---|---|\n"
        for k, v in data.items():
            if v is not None:
                report += f"| {k} | {v} |\n"

        report += "\n *Calculated values included. Consult a doctor.*"
        return report

    except Exception as e:
        return f"Analysis Error: {str(e)}"

print(" Dual-Mode CBC Tool Ready.")

Loading CBC Model and Scaler...
✅ CBC Model & Normalizer Ready.
 Dual-Mode CBC Tool Ready.


##  ROUTER LOGIC (Distinguish X-Ray vs CBC)

In [13]:
def classify_image_type(image_path):
    """
    Decides if an image is a CBC Report or an X-Ray based on text content.
    """
    try:
        # Fast scan for text
        text = pytesseract.image_to_string(Image.open(image_path)).upper()

        # Keywords found in blood reports but NOT in X-rays
        cbc_keywords = ["WBC", "HGB", "PLT", "NE%", "LY%", "BLOOD", "COUNT", "RESULT", "REFERENCE", "UNITS"]

        # Count how many keywords appear
        matches = sum(1 for word in cbc_keywords if word in text)

        # If we find 2 or more text matches, it's a Report. Otherwise, it's an X-Ray.
        if matches >= 2:
            return "CBC"
        else:
            return "XRAY"
    except:
        # If OCR fails (no text found), it is likely an image (X-Ray)
        return "XRAY"

print(" Router Logic Ready.")

✅ Router Logic Ready.


In [14]:

class UnslothLLM(LLM):
    """Custom wrapper for Unsloth FastLanguageModel to work with LangChain"""

    # Define these as fields to satisfy Pydantic
    model_obj: Any = Field(None, description="The Unsloth model object")
    tokenizer_obj: Any = Field(None, description="The tokenizer object")

    def __init__(self, model, tokenizer):
        super().__init__()
        self.model_obj = model
        self.tokenizer_obj = tokenizer

    @property
    def _llm_type(self) -> str:
        return "unsloth-llama"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # Prepare inputs
        inputs = self.tokenizer_obj([prompt], return_tensors="pt").to(self.model_obj.device)

        # Generate output
        outputs = self.model_obj.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.1, # Low temp for the Agent to be precise with tools
            use_cache=True,
            pad_token_id=self.tokenizer_obj.eos_token_id
        )

        # Decode
        response = self.tokenizer_obj.decode(outputs[0], skip_special_tokens=True)

        # Clean up response (remove the prompt echo if present)
        if prompt in response:
            response = response.split(prompt)[-1]

        return response.strip()

# Instantiate the wrapped LLM
llm = UnslothLLM(lora_model, tokenizer)
print(" LangChain LLM Wrapper created!")

✅ LangChain LLM Wrapper created!


### TOOL FUNCTIONS

In [15]:
# --- Tool 1: General Medical Advice (Unchanged, but ensures access to alpaca_prompt) ---
def medical_advice_func(query: str):
    # This logic now needs to handle your custom greetings/gratitude,
    # but since the LangGraph 'decide_tool' function is a simple router,
    # we'll let it execute the full LLM logic directly here.

    # We'll re-implement the LLM call using your preferred parameters
    instruction = system_instruction
    prompt = alpaca_prompt.format(instruction, query)

    inputs = tokenizer([prompt], return_tensors="pt").to(lora_model.device)
    outputs = lora_model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.2,
        top_p=0.8,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
    )
    res = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "### Response:" in res:
        return res.split("### Response:")[-1].strip()
    return res
# --- Tool 2: X-Ray Classifier (Base64 Fail-Safe Version) ---
def xray_diagnosis_func(image_path: str):
    """
    Runs DenseNet prediction, generates Grad-CAM heatmap,
    and returns the image embedded directly as text.
    """
    if not image_path or image_path == "None":
        return "Please upload an X-ray image first."

    try:
        # Load the image
        img_np = np.array(Image.open(image_path).convert('RGB'))

        # Get prediction
        idx, conf, probs = predict_single_image(image_path, model, val_transforms)
        predicted_class = class_names[idx]
        prob_dict = {class_names[i]: float(probs[i]) for i in range(len(class_names))}

        # Start formatting text
        txt = f"### 🩻 X-Ray Analysis Result\n"
        txt += f"**Prediction:** {predicted_class} | **Confidence:** {conf:.4f}\n"
        txt += f"**Probabilities:** {', '.join([f'{k}: {v:.2f}' for k, v in prob_dict.items()])}\n\n"
        txt += " *Always consult a doctor for a professional diagnosis.*"

        # --- Grad-CAM Logic (only for Pneumonia) ---
        if predicted_class == "PNEUMONIA":
            # Generate Heatmap
            target_layer = model.features[-1]
            gradcam = GradCAM(model, target_layer)
            input_tensor = val_transforms(Image.fromarray(img_np)).unsqueeze(0).to(device)
            heatmap = gradcam.generate_heatmap(input_tensor, class_idx=1)

            # Resize and Overlay
            heatmap = cv2.resize(heatmap, (img_np.shape[1], img_np.shape[0]))
            heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
            heatmap_color = cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB)
            overlay = np.uint8(0.4 * heatmap_color + 0.6 * img_np)

            #  ENCODE IMAGE TO BASE64 STRING
            # This converts the image pixels directly into a text string
            pil_img = Image.fromarray(overlay)
            buffered = BytesIO()
            pil_img.save(buffered, format="JPEG")
            img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

            # Create the Data URI
            img_data_uri = f"data:image/jpeg;base64,{img_str}"

            # Append to output using standard Markdown image syntax
            txt += f"\n\n--- Grad-CAM Visualization ---\n"
            txt += f"Localization of pneumonia (heatmap overlay): \n\n![Grad-CAM Visualization]({img_data_uri})"

        else:
             txt += "\n\n*No Grad-CAM heatmap generated (Prediction is not PNEUMONIA).*"

        return txt

    except Exception as e:
        return f" Error processing image for diagnosis: {str(e)}"
# Define the LangChain Tools (The array remains the same)
tools = [
    Tool(
        name="Medical_QA",
        func=medical_advice_func,
        description="Use this tool to answer general medical questions."
    ),
    Tool(
        name="XRay_Analyzer",
        func=xray_diagnosis_func,
        description="Use this tool ONLY for X-ray image analysis."
    )
]
print(" Tools updated with Grad-CAM logic.")

✅ Tools updated with Grad-CAM logic.


In [16]:
# --- Define the State for the Agent ---
class AgentState(TypedDict):
    input: str
    output: str
    steps: List[str]

# --- Decide which tool to use ---
def decide_tool(state: AgentState):
    query = state["input"].lower()

    if any(x in query for x in ["xray", "x-ray", "image", ".png", ".jpg", ".jpeg"]):
        tool_result = tools[1].func(state["input"])
        return {"output": tool_result, "steps": state["steps"] + ["Used XRay_Analyzer"]}

    # Otherwise use NLP medical model
    tool_result = tools[0].func(state["input"])
    return {"output": tool_result, "steps": state["steps"] + ["Used Medical_QA"]}

# --- Build graph ---
workflow = StateGraph(AgentState)
workflow.add_node("decide", decide_tool)
workflow.set_entry_point("decide")

# Compile
agent = workflow.compile()
print(" LangGraph Agent Ready!")


🔥 LangGraph Agent Ready!


## SMART CHATBOT

In [16]:
import gradio as gr
from PIL import Image

def chat_fn(message, history):
    # 1. Parse Input
    user_text = message.get("text", "").strip()
    user_files = message.get("files", [])
    user_image = None
    if isinstance(user_files, list) and len(user_files) > 0:
        user_image = user_files[0]

    # ==========================================
    #  CASE 1: IMAGE UPLOADED -> RUN ROUTER
    # ==========================================
    if user_image:
        # Save image locally
        image_path = "/content/user_upload_temp.jpg"
        Image.open(user_image).convert('RGB').save(image_path)

        #  CALL THE ROUTER (CBC OR X-RAY)
        image_type = classify_image_type(image_path)

        report = ""
        if image_type == "CBC":
            report = cbc_diagnosis_func(image_path)  # OCR + ML
        else:
            report = xray_diagnosis_func(image_path)  # DenseNet + GradCAM

        # Clean report for LLM (remove image Markdown)
        clean_report = report.split("![")[0][:1500]

        # --- SUB-CASE A: IMAGE + TEXT ---
        if user_text:
            combined_prompt = f"""
            Context: User uploaded a {image_type}.
            Automated Analysis:
            {clean_report}

            User Question: "{user_text}"

            Task: Answer the user's question using the analysis above.
            """
            llm_response = medical_advice_func(combined_prompt)
            return report + "\n\n---\n### 🤖 AI Comment:\n" + llm_response

        # --- SUB-CASE B: IMAGE ONLY ---
        auto_prompt = f"""
        A user uploaded a {image_type} for medical analysis.
        Below is the automated report:
        {clean_report}

        Task: Explain the findings in simple terms and suggest next steps.
        """
        llm_interpret = medical_advice_func(auto_prompt)
        return report + "\n\n---\n### 🤖 AI Interpretation:\n" + llm_interpret

    # ==========================================
    # 💬 CASE 2: TEXT ONLY -> CHECK GREETINGS OR RUN LLM
    # ==========================================
    if user_text:
        lower_text = user_text.lower()

        # --- 1. EXTENDED GREETINGS (English + Arabic) ---
        greetings = [
            "hi", "hello", "hey", "hi there", "hello there", "hey there",
            "good morning", "good afternoon", "good evening",
            "greetings", "yo", "what's up", "sup", "howdy",
            # Arabic formal
            "اهلا", "أهلا", "اهلا وسهلا", "أهلاً وسهلاً",
            "مرحبا", "مرحباً", "السلام عليكم", "وعليكم السلام",
            "صباح الخير", "مساء الخير",
            # Arabic casual slang
            "هاي", "هلو", "ازيك", "إزيك",
            "عامل ايه", "عاملة ايه", "اخبارك", "أخبارك",
            "كيف الحال", "شلونك", "الو", "ألو"
        ]

        if lower_text in greetings:
            return "Hello! I am your medical AI assistant. You can ask me any medical question."

        # --- 2. THANK YOU PHRASES (English + Arabic) ---
        gratitude_msgs = [
            "thank you", "thanks", "thanks a lot", "you helped me", "thank you very much",
            "شكرا", "شكرًا", "شكرا جزيلا", "متشكر", "متشكرة",
            "اشكرك", "تسلم", "ربنا يخليك", "كتر خيرك"
        ]

        if any(msg in lower_text for msg in gratitude_msgs):
            return "You are welcome! I'm here to help."

        # --- 3. MEDICAL QUESTION (Run LLM) ---
        return medical_advice_func(user_text)

    # ==========================================
    # NO INPUT
    # ==========================================
    return "Please upload an image or ask a medical question."

# ==============================
#  Launch the Chatbot UI
# ==============================
chatbot = gr.ChatInterface(
    fn=chat_fn,
    title="Medical Consultant",
    description="You can Upload an X-ray or a CBC blood report or Ask any Medical Question",
    multimodal=True,
    type="messages"
)

chatbot.launch(debug=True, share=True, allowed_paths=["/content"])

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f1e98bb55b2aad5733.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils

Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 3043, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2680694954.py", line 113, in <cell line: 0>
    chatbot.launch(debug=True, share=True, allowed_paths=["/content"])
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2950, in launch
    self.block_thread()
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 3047, in block_thread
    self.server.close()
  File "/usr/local/lib/python3.12/dist-packages/gradio/http_server.py", line 69, in close
    self.thread.join(timeout=5)
  File "/usr/lib/p

TypeError: object of type 'NoneType' has no len()